In [ ]:
import torch
from ultralytics import YOLO
import os
import cv2
import pandas as pd

In [ ]:

model=YOLO('Model/model ver.1/best.pt')

In [159]:
# 포크리프트 1
img1=cv2.imread('실험/image/vv2.jpg')
img2=cv2.imread('실험/image/231231_1600_Cam5_C2 (1758).jpg')
img3=cv2.imread('실험/image/포크부분.png')

In [154]:
result=model.predict(img1)
for r in result:
    print(r)


0: 384x640 1 forklift_w, 72.4ms
Speed: 3.7ms preprocess, 72.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to c:\Users\KDP-30\runs\detect\predict
1 label saved to c:\Users\KDP-30\runs\detect\predict\labels
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person_o', 1: 'person_x', 2: 'forklift_w', 3: 'forklift_a', 4: 'forklift_s', 5: 'forklift_d', 6: 'trolly'}
obb: None
orig_img: array([[[ 42,  40,  39],
        [ 31,  32,  30],
        [ 23,  24,  22],
        ...,
        [ 39,  52,  50],
        [ 39,  52,  50],
        [ 39,  52,  50]],

       [[ 36,  39,  37],
        [ 30,  33,  31],
        [ 24,  27,  25],
        ...,
        [ 39,  52,  50],
        [ 39,  52,  50],
        [ 39,  52,  50]],

       [[ 30,  39,  36],
        [ 26,  35,  32],
        [ 26,  31,  29],
        ...,
        [ 39,  52,  50],
        [ 39,  52,  50],
        [ 39,  

In [ ]:
result=model.predict(img2,save_txt=True,save=True)
for r in result:
    r=r.to_df()
r

In [ ]:
r.iloc[0,3] #dict로 출력

In [181]:
result=model.predict(img2,save_txt=True,save=True)
for r in result:
    DF=r.to_df()
    personDF=DF[(DF['class']==0)|(DF['class']==1)]
    carDF=DF.drop(personDF.index)
    print(r[3].boxes.xywh)



0: 384x640 3 person_os, 1 forklift_d, 101.1ms
Speed: 1.0ms preprocess, 101.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to c:\Users\KDP-30\runs\detect\predict
1 label saved to c:\Users\KDP-30\runs\detect\predict\labels
tensor([[652.2251, 687.2020, 127.6326,  65.5960]])


In [121]:
personDF

,name,class,confidence,box
0,person_o,0,0.92648,"{'x1': 932.78784, 'y1': 578.05078, 'x2': 1134...."
2,person_o,0,0.88679,"{'x1': 930.29053, 'y1': 241.26184, 'x2': 1039...."
3,person_o,0,0.86928,"{'x1': 588.40881, 'y1': 654.40399, 'x2': 716.0..."


In [123]:
carDF

,name,class,confidence,box
1,forklift_d,5,0.92355,"{'x1': 467.78302, 'y1': 197.39716, 'x2': 960.9..."


In [116]:
0 in list(DF['class'])

True

In [117]:
# 안전모 x
if 1 in list(DF['class']):
    print('안전모써!!')

In [172]:
# 사람vs 차
for c in carDF.box:
    for p in personDF.box:
        print(r.boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([5., 0.])
conf: tensor([0.9171, 0.8175])
data: tensor([[7.1157e+02, 3.0674e+02, 1.4440e+03, 8.9766e+02, 9.1709e-01, 5.0000e+00],
        [5.4984e+02, 3.0369e+02, 7.0931e+02, 6.6777e+02, 8.1753e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (1084, 1915)
shape: torch.Size([2, 6])
xywh: tensor([[1077.8075,  602.2028,  732.4714,  590.9177],
        [ 629.5732,  485.7299,  159.4682,  364.0724]])
xywhn: tensor([[0.5628, 0.5555, 0.3825, 0.5451],
        [0.3288, 0.4481, 0.0833, 0.3359]])
xyxy: tensor([[ 711.5718,  306.7440, 1444.0432,  897.6616],
        [ 549.8391,  303.6937,  709.3073,  667.7661]])
xyxyn: tensor([[0.3716, 0.2830, 0.7541, 0.8281],
        [0.2871, 0.2802, 0.3704, 0.6160]])


In [145]:
carDF

,name,class,confidence,box
1,forklift_d,5,0.92355,"{'x1': 467.78302, 'y1': 197.39716, 'x2': 960.9..."


In [158]:
result=model.predict(img2,save_txt=True,save=True)
ratio=1.7
for r in result:
    DF=r.to_df()
    personDF=DF[(DF['class']==0)|(DF['class']==1)]  # 사람 관련 DF
    carDF=DF.drop(personDF.index)       # 차량 관련 DF
    if len(personDF)*len(carDF):        # 차와 사람이 둘다있을경우만 진행
        for c_idx in carDF.index:
            w,a,s,d=1,1,1,1
            c=carDF.loc[c_idx,'box']
            if carDF.loc[c_idx,'class']==2:
                w=ratio
            elif carDF.loc[c_idx,'class']==3:
                a=ratio
            elif carDF.loc[c_idx,'class']==4:
                s=ratio
            elif carDF.loc[c_idx,'class']==3:
                d=ratio
            for p in personDF.box:
                if ((c['x1']*a<=p['x1']<=c['x2']*d) or (c['x1']*a<=p['x2']<=c['x2']*d)) and ((c['y1']*w<=p['y1']<=c['y2']*s) or (c['y1']*w<=p['y2']<=c['y2']*s)) :
                    print('사고났다!')


0: 384x640 3 person_os, 1 forklift_d, 113.3ms
Speed: 2.0ms preprocess, 113.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to c:\Users\KDP-30\runs\detect\predict
1 label saved to c:\Users\KDP-30\runs\detect\predict\labels
사고났다!
사고났다!


In [183]:
result=model.predict(img3,save_txt=True,save=True)
ratio=0.7
for r in result:
    DF=r.to_df()
    personDF=DF[(DF['class']==0)|(DF['class']==1)]  # 사람 관련 DF
    carDF=DF.drop(personDF.index)       # 차량 관련 DF
    #carDF['class']=3
    if len(personDF)*len(carDF):        # 차와 사람이 둘다있을경우만 진행
        for c_idx in carDF.index:
            w,a,s,d=1,1,1,1
            c=carDF.loc[c_idx,'box']
            if carDF.loc[c_idx,'class']==2:
                w=ratio
            elif carDF.loc[c_idx,'class']==3:
                a=ratio
            elif carDF.loc[c_idx,'class']==4:
                s=ratio
            elif carDF.loc[c_idx,'class']==3:
                d=ratio
            wid=c['x2']-c['x1']
            print(wid==r[c_idx].boxes.xywh[2])
            hei=c['y2']-c['y1']
            for p in personDF.box:
                if ((c['x1']-wid*a<=p['x1']<=c['x2']+wid*d) or (c['x1']-wid*a<=p['x2']<=c['x2']+wid*d)) and ((c['y1']-hei*w<=p['y1']<=c['y2']+hei*s) or (c['y1']-hei*w<=p['y2']<=c['y2']+hei*s)) :
                    print('사고났다!')


0: 384x640 1 person_o, 1 forklift_d, 95.9ms
Speed: 2.0ms preprocess, 95.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to c:\Users\KDP-30\runs\detect\predict
1 label saved to c:\Users\KDP-30\runs\detect\predict\labels


IndexError: index 2 is out of bounds for dimension 0 with size 1

In [167]:
carDF

,name,class,confidence,box
0,forklift_d,3,0.91709,"{'x1': 711.57184, 'y1': 306.74396, 'x2': 1444...."


In [ ]:
def car_accident(img,ratio=0.7):
    result=model.predict(img,save_txt=True,save=True)
    for r in result:
        DF=r.to_df()
        personDF=DF[(DF['class']==0)|(DF['class']==1)]  # 사람 관련 DF
        carDF=DF.drop(personDF.index)       # 차량 관련 DF
        w,a,s,d=0                           # 4방향 가중치 초기화
        if len(personDF)*len(carDF):        # 차와 사람이 둘다있을경우만 진행
            for c_idx in carDF.index:
                c=carDF.loc[c_idx,'box']
                if carDF.loc[c_idx,'class']==2:            # w 방향 가중치
                    w=ratio
                elif carDF.loc[c_idx,'class']==3:           # a 방향 가중치
                    a=ratio
                elif carDF.loc[c_idx,'class']==4:           # s 방향 가중치
                    s=ratio
                elif carDF.loc[c_idx,'class']==3:           # d 방향 가중치
                    d=ratio
                wid=c['x2']-c['x1']         # 차너비
                hei=c['y2']-c['y1']         # 차높이
                for p in personDF.box:
                    if ((c['x1']-wid*a<=p['x1']<=c['x2']+wid*d) or (c['x1']-wid*a<=p['x2']<=c['x2']+wid*d)) and ((c['y1']-hei*w<=p['y1']<=c['y2']+hei*s) or (c['y1']-hei*w<=p['y2']<=c['y2']+hei*s)) :
                        print('사고났다!')